<img src="https://afs-services.com/wp-content/uploads/2025/09/dp_banner.png" alt="RV Banner" style="width: 100%; height: auto; display: block;">

## 

In [1]:
from src.bdptplt import DataPlatformDB
import pandas as pd
from datetime import datetime as dt, timedelta, time
import pytz
import numpy as np

In [8]:
# Use nest_asyncio to patch the loop and avoid asyncio errors inside Notebook
import nest_asyncio
nest_asyncio.apply()

In [2]:
%load_ext dotenv
%dotenv .env -o

#### Steps to create new table in DDBB
**Notes:**
- The table must have a primary key named `dtime`.

**Steps**

0. Go to `prebd/tslibv2.py`.  
1. Add the table in the `TableCols` dictionary.  
2. Add data in the `timescale_config` dictionary.  
3. If `dtime` is of type `date`, add the table name in the `tabtype_date` list. 
4. Assign primary keys using the `get_primary_key` function.  
5. Assign the type of each column in the `createdbifneeded` function.  
   - **IMPORTANT:** Each column name in the entire database can only have one type.  
   - **NOTE:** If it is a numeric type, you don’t need to add it.  
6. Add the compression type in `enable_compression` *(OPTIONAL).*  
7. Run the upserter to save data into the table.  

#### Create DataPlarformDB instance

In [ ]:
db = DataPlatformDB()

### Insert data into BBDD

Generate a random t2t df:

In [18]:
import random

def create_dummy_df(n, contract_name):
    start_date = dt.combine(dt.now().date(), time(9, 0))  # Market opens at 9:00 AM

    # Generate random datetime values during the trading day
    dtime = [start_date + timedelta(seconds=random.randint(0, 32400)) for _ in range(n)]
    dtime.sort()  # Optional: sort by time

    # Simulate a starting price and small random walk (to mimic market noise)
    price = [100 + random.gauss(0, 0.5)]  # start around 100 with small volatility
    for _ in range(1, n):
        price.append(price[-1] + random.gauss(0, 0.5))  # continue random walk

    # Simulate trade sizes (e.g., integers from 1 to 1000 shares/contracts)
    size = [random.randint(1, n) for _ in range(n)]

    # Create DataFrame
    df = pd.DataFrame({
        'dtime': dtime,
        'contract_id': range(0, n),
        'contract_name' : contract_name,
        'price': price,
        'size': size,
    })

    print(df.head())

    return df.copy()

In [ ]:
df1 = create_dummy_df(100, 'dummy_contract')

Insert data

In [ ]:
db.upserter(df=df1, tabtype="prob", instrument='PRB')

### Load data from BBDD

Create LoaderInput instance

In [10]:
LoaderInput = DataPlatformDB.loader_input
input_obj = LoaderInput(table="PRB_prob")

Load data

In [ ]:
db.loader([input_obj])